# Example 6 - Test

In [1]:
from CircuitCalculator.Circuit.circuit import Circuit, transform_circuit
import CircuitCalculator.Circuit.components as ccp
from CircuitCalculator.Circuit.solution import DCSolution
import numpy as np

In [2]:
R1, R2, R3, R4, R5 = 10, 20, 30, 40, 50
Vs1, Vs2 = 1, 2
circuit = Circuit([
    ccp.dc_voltage_source(V=Vs1, id='Vs1', nodes=('0', '1')),
    ccp.dc_voltage_source(V=Vs2, id='Vs2', nodes=('3', '2')),
    ccp.resistor(R=R1, id='R1', nodes=('1', '2')),
    ccp.resistor(R=R2, id='R2', nodes=('2', '0')),
    ccp.resistor(R=R3, id='R3', nodes=('2', '3')),
    ccp.resistor(R=R4, id='R4', nodes=('3', '4')),
    ccp.resistor(R=R5, id='R5', nodes=('4', '0')),
    ccp.ground(nodes=('0', ))
])
network = transform_circuit(circuit, w=0)

In [3]:
solution = DCSolution(circuit)
print(f'phi 1={solution.get_potential("1"):2.2f}V')
print(f'phi 2={solution.get_potential("2"):2.2f}V')
print(f'phi 3={solution.get_potential("3"):2.2f}V')
print(f'phi 4={solution.get_potential("4"):2.2f}V')

phi 1=-1.00V
phi 2=-2.21V
phi 3=-0.21V
phi 4=0.54V


In [4]:
G1, G2, G3, G4, G5 = 1/R1, 1/R2, 1/R3, 1/R4, 1/R5
A = np.array([
    # [ G2+G5,    0,      -G2,     0,   -G5,  1,  0],
    [     0,   G1,      -G1,     0,     0, -1,  0],
    [    -G2, -G1, G1+G2+G3,   -G3,     0,  0, -1],
    [      0,   0,      -G3, G3+G4,   -G4,  0,  1],
    [    -G5,   0,        0,   -G4, G4+G5,  0,  0],
    [     -1,   1,        0,     0,     0,  0,  0],
    [      0,   0,        1,    -1,     0,  0,  0],
    [      1,   0,        0,     0,     0,  0,  0]
])
print(np.linalg.cond(A))

Is = np.array([0, 0, 0, 0, Vs1, Vs2, 0])

x = np.linalg.solve(A, Is)
phi = x[:5]
i = x[5:]
print(phi)
print(i)

42.06949542621331
[-0.          1.          0.75862069 -1.24137931 -0.68965517]
[0.02413793 0.08045977]


In [5]:
G1, G2, G3, G4, G5 = 1/R1, 1/R2, 1/R3, 1/R4, 1/R5
A = np.array([
    [ G1,      -G1,     0,     0, -1,  0],
    [-G1, G1+G2+G3,   -G3,     0,  0, -1],
    [  0,      -G3, G3+G4,   -G4,  0,  1],
    [  0,        0,   -G4, G4+G5,  0,  0],
    [  1,        0,     0,     0,  0,  0],
    [  0,        1,    -1,     0,  0,  0],
])
print(np.linalg.cond(A))

Is = np.array([0, 0, 0, 0, Vs1, Vs2])

x = np.linalg.solve(A, Is)
phi = x[:4]
i = x[4:]
print(phi)
print(i)

38.87388384698125
[ 1.          0.75862069 -1.24137931 -0.68965517]
[0.02413793 0.08045977]


In [6]:
from CircuitCalculator.Network.network import Network
from CircuitCalculator.Network.elements import is_ideal_voltage_source
import CircuitCalculator.Network.NodalAnalysis.label_mapping as map
from CircuitCalculator.Network.NodalAnalysis.node_analysis import node_admittance_matrix
import itertools
def create_voltage_source_incidence_matrix(network: Network, node_mapper: map.NetworkMapper = map.default_node_mapper, source_mapper: map.SourceIndexMapper = map.default_source_mapper) -> np.ndarray:
    def voltage_source_direction(voltage_source: str, node: str) -> int:
        if network[voltage_source].node1 == node:
            return 1
        if network[voltage_source].node2 == node:
            return -1
        return 0
    node_index = node_mapper(network)
    vs_index = map.filter_mapping(source_mapper(network), lambda x: is_ideal_voltage_source(network[x].element))
    B = np.zeros((node_index.N, vs_index.N))
    for node, vs in itertools.product(node_index.keys, vs_index.keys):
        B[node_index[node], vs_index[vs]] = voltage_source_direction(vs, node)
    return B

In [7]:
def create_voltage_source_potential_incidence_matrix(network, node_mapper: map.NetworkMapper = map.default_node_mapper, source_mapper: map.SourceIndexMapper = map.default_source_mapper) -> np.ndarray:
    def current_direction(voltage_source: str, node: str) -> int:
        if network[voltage_source].node1 == node:
            return -1
        if network[voltage_source].node2 == node:
            return 1
        return 0
    node_index = node_mapper(network)
    vs_index = map.filter_mapping(source_mapper(network), lambda x: is_ideal_voltage_source(network[x].element))
    C = np.zeros((vs_index.N, node_index.N))
    for vs, node in itertools.product(vs_index.keys, node_index.keys):
        C[vs_index[vs], node_index[node]] = current_direction(vs, node)
    return C


In [8]:
def create_coefficient_matrix(network: Network, node_mapper: map.NetworkMapper = map.default_node_mapper, source_mapper: map.SourceIndexMapper = map.default_source_mapper) -> np.ndarray:
    Y = node_admittance_matrix(network, node_mapper)
    B = create_voltage_source_incidence_matrix(network, node_mapper, source_mapper)
    C = create_voltage_source_potential_incidence_matrix(network, node_mapper, source_mapper)
    Z = np.zeros((B.shape[1], C.shape[0]))
    return np.vstack((np.hstack((Y, B)), np.hstack((C, Z))))

In [10]:
A = create_coefficient_matrix(network)

Is = np.array([0, 0, 0, 0, Vs1, Vs2])

x = np.linalg.solve(A, Is)
phi = x[:4]
i = x[4:]
print(phi.real)
print(i.real)


[ 1.          0.75862069 -1.24137931 -0.68965517]
[0.02413793 0.08045977]
